In [1]:
import py_vncorenlp

In [2]:
import os

ABSOLUTE_MODEL_PATH = os.path.abspath("vncorenlp")
print("Using model path:", ABSOLUTE_MODEL_PATH)

Using model path: d:\School\4thYear\2ndSemester\NLP\Comment-Sentiment-Analysis\attribute_extractor\vncorenlp


In [3]:
# Khởi tạo model từ thư mục local
model = py_vncorenlp.VnCoreNLP(save_dir=ABSOLUTE_MODEL_PATH, max_heap_size="-Xmx2g")

In [8]:
example="bao lâu"

In [9]:
model.word_segment(example)

['bao_lâu']

### Các chức năng mà model hỗ trợ

In [4]:
model.annotators

['wseg', 'pos', 'ner', 'parse']

In [5]:
# Dùng thử
text = "Cũng tạm. Hàng trưng bày có bị xước màn hình rồi nhưng mọi thứ dùng ổn. Pin hơi đuối. Tóm lại giá rẻ hơn mọi chỗ, máy zin all, chế độ bảo hành đổi trả kém hơn chỗ khác. OK"
result = model.annotate_text(text)
model.print_out(result)

1	Cũng	R	O	2	amod
2	tạm	A	O	0	root
3	.	CH	O	2	punct

1	Hàng	N	O	3	sub
2	trưng_bày	V	O	1	nmod
3	có	V	O	0	root
4	bị	V	O	3	vmod
5	xước	V	O	4	vmod
6	màn_hình	N	O	5	dob
7	rồi	R	O	5	adv
8	nhưng	C	O	3	coord
9	mọi	L	O	10	det
10	thứ	N	O	11	sub
11	dùng	V	O	8	conj
12	ổn	A	O	11	vmod
13	.	CH	O	3	punct

1	Pin	Np	O	3	sub
2	hơi	R	O	3	amod
3	đuối	A	O	0	root
4	.	CH	O	3	punct

1	Tóm_lại	X	O	3	x
2	giá	N	O	3	sub
3	rẻ	A	O	0	root
4	hơn	A	O	3	amod
5	mọi	L	O	6	det
6	chỗ	N	O	3	amod
7	,	CH	O	6	punct
8	máy	N	O	6	nmod
9	zin	V	O	8	nmod
10	all	N	O	8	nmod
11	,	CH	O	6	punct
12	chế_độ	N	O	6	nmod
13	bảo_hành	V	O	12	nmod
14	đổi	V	O	13	vmod
15	trả	V	O	14	vmod
16	kém	A	O	15	vmod
17	hơn	A	O	16	amod
18	chỗ	N	O	15	dob
19	khác	A	O	18	nmod
20	.	CH	O	3	punct

1	OK	Nb	O	0	root



In [6]:
result

{0: [{'index': 1,
   'wordForm': 'Cũng',
   'posTag': 'R',
   'nerLabel': 'O',
   'head': 2,
   'depLabel': 'amod'},
  {'index': 2,
   'wordForm': 'tạm',
   'posTag': 'A',
   'nerLabel': 'O',
   'head': 0,
   'depLabel': 'root'},
  {'index': 3,
   'wordForm': '.',
   'posTag': 'CH',
   'nerLabel': 'O',
   'head': 2,
   'depLabel': 'punct'}],
 1: [{'index': 1,
   'wordForm': 'Hàng',
   'posTag': 'N',
   'nerLabel': 'O',
   'head': 3,
   'depLabel': 'sub'},
  {'index': 2,
   'wordForm': 'trưng_bày',
   'posTag': 'V',
   'nerLabel': 'O',
   'head': 1,
   'depLabel': 'nmod'},
  {'index': 3,
   'wordForm': 'có',
   'posTag': 'V',
   'nerLabel': 'O',
   'head': 0,
   'depLabel': 'root'},
  {'index': 4,
   'wordForm': 'bị',
   'posTag': 'V',
   'nerLabel': 'O',
   'head': 3,
   'depLabel': 'vmod'},
  {'index': 5,
   'wordForm': 'xước',
   'posTag': 'V',
   'nerLabel': 'O',
   'head': 4,
   'depLabel': 'vmod'},
  {'index': 6,
   'wordForm': 'màn_hình',
   'posTag': 'N',
   'nerLabel': 'O',
   

### Tính năng tokenize + segment

In [7]:
model.word_segment(
    "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."
)

['Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .',
 'Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .']

### Bài toán trích xuất đánh giá cho từng thuộc tính trong comment

In [10]:
from typing import List, Dict, Any, Optional
import json  # Để in đẹp hơn

In [12]:
def extract_attributes_from_dependency_v3(
    parsed_sentence: List[Dict[str, Any]],
) -> List[Dict[str, Any]]:
    """
    Trích xuất các cặp (Danh từ, Thuộc tính) và các trạng từ bổ nghĩa
    từ kết quả dependency parsing của VnCoreNLP (Version 3).

    Xử lý các trường hợp:
    1. Tính từ (A) là 'nmod' của Danh từ (N/P). (vd: nhân viên nhiệt tình)
    2. Tính từ (A) là 'vmod' của Động từ (V), và tìm chủ ngữ (sub) của Động từ đó. (vd: pin sạc nhanh)
    3. Thuộc tính (A, V đặc biệt) có chủ ngữ (sub) hoặc bổ ngữ trực tiếp (dob) là Danh từ (N/P). (vd: loa tạm, giá cao)

    Tìm Trạng từ (R) bổ nghĩa cho thuộc tính (depLabel 'adv' hoặc 'amod').

    Args:
        parsed_sentence: List các dictionary, mỗi dict chứa thông tin 1 từ.

    Returns:
        Một list các dictionary, mỗi dict có dạng:
        {
            'noun': wordForm của danh từ được mô tả,
            'noun_index': index của danh từ,
            'attribute': wordForm của từ thuộc tính,
            'attribute_index': index của từ thuộc tính,
            'attribute_pos': posTag của từ thuộc tính,
            'modifiers': list các wordForm của trạng từ bổ nghĩa cho thuộc tính
        }
    """
    if not parsed_sentence:
        return []

    word_dict = {word["index"]: word for word in parsed_sentence}
    extracted_pairs = []
    processed_attribute_indices = set()  # Tránh xử lý lại cùng 1 thuộc tính

    # --- Helper Functions ---
    def find_modifiers(attr_node: Dict[str, Any]) -> List[str]:
        """Tìm các trạng từ (R) bổ nghĩa trực tiếp cho node thuộc tính (adv hoặc amod)."""
        mods = []
        attr_idx = attr_node.get("index")
        for word in word_dict.values():
            if (
                word.get("head") == attr_idx
                and word.get("posTag") == "R"
                and word.get("depLabel") in ("adv", "amod")
            ):
                mods.append(word.get("wordForm", ""))
        return mods

    def find_direct_subject_object_noun(
        node: Dict[str, Any],
    ) -> Optional[Dict[str, Any]]:
        """Tìm chủ ngữ (sub) hoặc bổ ngữ trực tiếp (dob) là N/P của node này."""
        node_idx = node.get("index")
        for word in word_dict.values():
            if word.get("head") == node_idx and word.get("posTag", "").startswith(
                ("N", "P")
            ):
                if word.get("depLabel") in ("sub", "dob"):
                    # Kiểm tra xem noun này có thuộc trường hợp nmod (đã xử lý ở ưu tiên 1)
                    has_direct_adj_mod = False
                    for w_check in word_dict.values():
                        if (
                            w_check.get("head") == word.get("index")
                            and w_check.get("depLabel") == "nmod"
                            and w_check.get("posTag") == "A"
                        ):
                            has_direct_adj_mod = True
                            break
                    if not has_direct_adj_mod:
                        return word
        return None

    # --- Main Logic ---
    # Duyệt qua từng từ để tìm các từ thuộc tính tiềm năng
    for word in parsed_sentence:
        word_idx = word.get("index")
        if word_idx in processed_attribute_indices:
            continue

        related_noun_node = None
        attribute_node = None

        # --- Ưu tiên 1: Tính từ (A) bổ nghĩa trực tiếp cho danh từ (nmod) ---
        if word.get("posTag") == "A" and word.get("depLabel") == "nmod":
            head_idx = word.get("head")
            if head_idx and head_idx in word_dict:
                head_node = word_dict[head_idx]
                if head_node.get("posTag", "").startswith(("N", "P")):
                    attribute_node = word
                    related_noun_node = head_node
                    processed_attribute_indices.add(word_idx)

        # --- Ưu tiên 2: Tính từ (A) là 'vmod' của Động từ (V), tìm chủ ngữ (sub) của V đó ---
        if (
            not attribute_node
            and word.get("posTag") == "A"
            and word.get("depLabel") == "vmod"
        ):
            head_idx = word.get("head")  # Index của Verb
            if head_idx and head_idx in word_dict:
                verb_node = word_dict[head_idx]
                if verb_node.get("posTag") == "V":
                    # Tìm chủ ngữ (sub) của Verb này
                    for potential_noun in word_dict.values():
                        if (
                            potential_noun.get("head") == head_idx
                            and potential_noun.get("depLabel") == "sub"
                            and potential_noun.get("posTag", "").startswith(("N", "P"))
                        ):
                            attribute_node = word
                            related_noun_node = potential_noun
                            processed_attribute_indices.add(word_idx)
                            break  # Tìm thấy chủ ngữ thì dừng

        # --- Ưu tiên 3: Tìm các thuộc tính khác (A, V đặc biệt) và noun liên quan (sub/dob của chính nó) ---
        if not attribute_node:
            is_potential_attribute = False
            if word.get("posTag") == "A":  # Tính từ làm root hoặc predicate khác
                is_potential_attribute = True
            elif word.get("posTag") == "V" and word.get("wordForm") in (
                "bị",
                "được",
                "hài_lòng",
            ):  # Động từ trạng thái
                is_potential_attribute = True

            if is_potential_attribute:
                # Tìm chủ ngữ (sub) hoặc bổ ngữ (dob) trực tiếp của thuộc tính này
                noun_node = find_direct_subject_object_noun(word)
                if noun_node:
                    attribute_node = word
                    related_noun_node = noun_node
                    processed_attribute_indices.add(word_idx)

        # Nếu tìm thấy cặp (noun, attribute)
        if related_noun_node and attribute_node:
            modifier_list = find_modifiers(attribute_node)
            extracted_pairs.append(
                {
                    "noun": related_noun_node.get("wordForm", ""),
                    "noun_index": related_noun_node.get("index"),
                    "attribute": attribute_node.get("wordForm", ""),
                    "attribute_index": attribute_node.get("index"),
                    "attribute_pos": attribute_node.get("posTag", ""),
                    "modifiers": modifier_list,
                }
            )

    # Lọc kết quả cuối cùng nếu cần (ví dụ: đảm bảo noun không trùng lặp quá nhiều)
    final_results = extracted_pairs

    return final_results

In [13]:
def extract_attributes_from_dependency_v4(
    parsed_sentence: List[Dict[str, Any]],
) -> List[Dict[str, Any]]:
    """
    Trích xuất các cặp (Danh từ, Thuộc tính) và các trạng từ bổ nghĩa (Version 4).

    Xử lý các trường hợp:
    1. Tính từ (A) là 'nmod' của Danh từ (N/P). (vd: nhân viên nhiệt tình)
    2. Tính từ (A) là 'vmod' của Động từ (V), và tìm chủ ngữ (sub) của Động từ đó. (vd: pin sạc nhanh)
    3. (Mới) Tính từ (A) và Động từ (V_link) là siblings (cùng head), tìm chủ ngữ (sub) của V_link. (vd: pin dùng lâu - khi parse bị rời rạc)
    4. Thuộc tính (A, V đặc biệt) có chủ ngữ (sub) hoặc bổ ngữ trực tiếp (dob) là Danh từ (N/P). (vd: loa tạm, giá cao)

    Tìm Trạng từ (R) bổ nghĩa cho thuộc tính (depLabel 'adv' hoặc 'amod').
    """
    if not parsed_sentence:
        return []

    word_dict = {word["index"]: word for word in parsed_sentence}
    extracted_pairs = []
    processed_attribute_indices = set()  # Tránh xử lý lại cùng 1 thuộc tính
    processed_noun_indices = set()  # Tránh noun bị gán nhiều thuộc tính không liên quan

    # --- Helper Functions (giữ nguyên từ v3) ---
    def find_modifiers(attr_node: Dict[str, Any]) -> List[str]:
        mods = []
        attr_idx = attr_node.get("index")
        for word in word_dict.values():
            if (
                word.get("head") == attr_idx
                and word.get("posTag") == "R"
                and word.get("depLabel") in ("adv", "amod")
            ):
                mods.append(word.get("wordForm", ""))
        return mods

    def find_direct_subject_object_noun(
        node: Dict[str, Any],
    ) -> Optional[Dict[str, Any]]:
        node_idx = node.get("index")
        for word in word_dict.values():
            if word.get("head") == node_idx and word.get("posTag", "").startswith(
                ("N", "P")
            ):
                if word.get("depLabel") in ("sub", "dob"):
                    has_direct_adj_mod = False
                    for w_check in word_dict.values():
                        if (
                            w_check.get("head") == word.get("index")
                            and w_check.get("depLabel") == "nmod"
                            and w_check.get("posTag") == "A"
                        ):
                            has_direct_adj_mod = True
                            break
                    if not has_direct_adj_mod:
                        return word
        return None

    # --- End Helper Functions ---

    # --- Main Logic ---
    # Duyệt qua từng từ để tìm các từ thuộc tính tiềm năng
    for word in parsed_sentence:
        word_idx = word.get("index")
        if word_idx in processed_attribute_indices:
            continue

        related_noun_node = None
        attribute_node = None

        # --- Ưu tiên 1: Tính từ (A) bổ nghĩa trực tiếp cho danh từ (nmod) ---
        if word.get("posTag") == "A" and word.get("depLabel") == "nmod":
            head_idx = word.get("head")
            if head_idx and head_idx in word_dict:
                head_node = word_dict[head_idx]
                if (
                    head_node.get("posTag", "").startswith(("N", "P"))
                    and head_idx not in processed_noun_indices
                ):
                    attribute_node = word
                    related_noun_node = head_node
                    processed_attribute_indices.add(word_idx)
                    processed_noun_indices.add(
                        head_idx
                    )  # Đánh dấu noun đã có attribute trực tiếp

        # --- Ưu tiên 2: Tính từ (A) là 'vmod' của Động từ (V), tìm chủ ngữ (sub) của V đó ---
        if (
            not attribute_node
            and word.get("posTag") == "A"
            and word.get("depLabel") == "vmod"
        ):
            head_idx = word.get("head")  # Index của Verb
            if head_idx and head_idx in word_dict:
                verb_node = word_dict[head_idx]
                if verb_node.get("posTag") == "V":
                    for potential_noun in word_dict.values():
                        if (
                            potential_noun.get("head") == head_idx
                            and potential_noun.get("depLabel") == "sub"
                            and potential_noun.get("posTag", "").startswith(("N", "P"))
                            and potential_noun.get("index")
                            not in processed_noun_indices
                        ):
                            attribute_node = word
                            related_noun_node = potential_noun
                            processed_attribute_indices.add(word_idx)
                            processed_noun_indices.add(potential_noun.get("index"))
                            break

        # --- Ưu tiên 2b (Mới): Tính từ (A) và Động từ (V_link) là siblings, tìm chủ ngữ (sub) của V_link ---
        if (
            not attribute_node and word.get("posTag") == "A" and word.get("head")
        ):  # Chỉ xét A có head
            attr_head_idx = word.get("head")
            # Tìm động từ V_link có cùng head với A
            for potential_verb_idx, potential_verb_node in word_dict.items():
                if (
                    potential_verb_idx != word_idx
                    and potential_verb_node.get("posTag") == "V"
                    and potential_verb_node.get("head") == attr_head_idx
                ):  # Cùng head
                    # Tìm chủ ngữ (sub) của potential_verb_node
                    for potential_noun in word_dict.values():
                        if (
                            potential_noun.get("head") == potential_verb_idx
                            and potential_noun.get("depLabel") == "sub"
                            and potential_noun.get("posTag", "").startswith(("N", "P"))
                            and potential_noun.get("index")
                            not in processed_noun_indices
                        ):
                            # Tìm thấy cấu trúc N -> V_link và A -> cùng head
                            attribute_node = word
                            related_noun_node = potential_noun
                            processed_attribute_indices.add(word_idx)
                            processed_noun_indices.add(potential_noun.get("index"))
                            break  # Dừng tìm noun
                    if attribute_node:
                        break  # Dừng tìm verb

        # --- Ưu tiên 3: Tìm các thuộc tính khác (A, V đặc biệt) và noun liên quan (sub/dob của chính nó) ---
        if not attribute_node:
            is_potential_attribute = False
            if word.get("posTag") == "A":
                is_potential_attribute = True
            elif word.get("posTag") == "V" and word.get("wordForm") in (
                "bị",
                "được",
                "hài_lòng",
            ):
                is_potential_attribute = True

            if is_potential_attribute:
                noun_node = find_direct_subject_object_noun(word)
                if noun_node and noun_node.get("index") not in processed_noun_indices:
                    attribute_node = word
                    related_noun_node = noun_node
                    processed_attribute_indices.add(word_idx)
                    processed_noun_indices.add(noun_node.get("index"))

        # Nếu tìm thấy cặp (noun, attribute)
        if related_noun_node and attribute_node:
            modifier_list = find_modifiers(attribute_node)
            extracted_pairs.append(
                {
                    "noun": related_noun_node.get("wordForm", ""),
                    "noun_index": related_noun_node.get("index"),
                    "attribute": attribute_node.get("wordForm", ""),
                    "attribute_index": attribute_node.get("index"),
                    "attribute_pos": attribute_node.get("posTag", ""),
                    "modifiers": modifier_list,
                }
            )

    final_results = extracted_pairs
    return final_results

In [14]:
def extract_attributes_from_all_segments_of_sentence(sentence):
    """
    Trích xuất các thuộc tính từ tất cả các phân đoạn của câu.
    """
    all_attributes = []
    parsed_sentence_data = model.annotate_text(sentence)
    for i in range(len(parsed_sentence_data)):
        sentence_data = parsed_sentence_data[i]
        attributes = extract_attributes_from_dependency_v4(sentence_data)
        if attributes:  # Chỉ thêm vào nếu có kết quả
            all_attributes.extend(attributes)
    return all_attributes

In [15]:
example1 = "Loa cũng tạm, âm thanh nghe cũng được, giá hơi cao nha :)"
example2 = "Được cái nhân viên nhiệt tình, giao hàng nhanh."
example3 = "Màn hình chụp đẹp, máy chạy mượt, pin dùng lâu"
example4 = "Nhân viên ở đây rất nhiệt tình, tôi hài lòng với dịch vụ. Điện thoại giá cả hợp lý, chất lượng tốt."
example5 = "nhân viên thân thiện tư vấn nhiệt tình, cảm ơn nhân viên đmx Quảng Tùng"
example6 = "sản phẩm sài ok lắm"
example7="điện thoại dùng mượt, chụp hình đẹp, pin dùng mãi chả thấy hết để sạc, rất hài lòng. Ai đang định mua thì chọn luôn không phải lăn tăn nhé"

In [16]:
print(example1)
print("\n---\n")
extract_attributes_from_all_segments_of_sentence(example1)

Loa cũng tạm, âm thanh nghe cũng được, giá hơi cao nha :)

---



[{'noun': 'Loa',
  'noun_index': 1,
  'attribute': 'tạm',
  'attribute_index': 3,
  'attribute_pos': 'A',
  'modifiers': ['cũng']},
 {'noun': 'âm_thanh',
  'noun_index': 5,
  'attribute': 'được',
  'attribute_index': 8,
  'attribute_pos': 'V',
  'modifiers': ['cũng']},
 {'noun': 'giá',
  'noun_index': 10,
  'attribute': 'cao',
  'attribute_index': 12,
  'attribute_pos': 'A',
  'modifiers': ['hơi']}]

In [17]:
print(example2)
print("\n---\n")
extract_attributes_from_all_segments_of_sentence(example2)

Được cái nhân viên nhiệt tình, giao hàng nhanh.

---



[{'noun': 'nhân_viên',
  'noun_index': 2,
  'attribute': 'nhiệt_tình',
  'attribute_index': 3,
  'attribute_pos': 'A',
  'modifiers': []},
 {'noun': 'hàng',
  'noun_index': 6,
  'attribute': 'nhanh',
  'attribute_index': 7,
  'attribute_pos': 'A',
  'modifiers': []}]

In [18]:
print(example3)
print("\n---\n")
extract_attributes_from_all_segments_of_sentence(example3)

Màn hình chụp đẹp, máy chạy mượt, pin dùng lâu

---



[{'noun': 'Màn_hình',
  'noun_index': 1,
  'attribute': 'đẹp',
  'attribute_index': 3,
  'attribute_pos': 'A',
  'modifiers': []},
 {'noun': 'máy',
  'noun_index': 5,
  'attribute': 'mượt',
  'attribute_index': 7,
  'attribute_pos': 'A',
  'modifiers': []},
 {'noun': 'pin',
  'noun_index': 9,
  'attribute': 'lâu',
  'attribute_index': 11,
  'attribute_pos': 'A',
  'modifiers': []}]

In [19]:
print(example4)
print("\n---\n")
extract_attributes_from_all_segments_of_sentence(example4)

Nhân viên ở đây rất nhiệt tình, tôi hài lòng với dịch vụ. Điện thoại giá cả hợp lý, chất lượng tốt.

---



[{'noun': 'Nhân_viên',
  'noun_index': 1,
  'attribute': 'nhiệt_tình',
  'attribute_index': 5,
  'attribute_pos': 'A',
  'modifiers': ['rất']},
 {'noun': 'tôi',
  'noun_index': 7,
  'attribute': 'hài_lòng',
  'attribute_index': 8,
  'attribute_pos': 'V',
  'modifiers': []},
 {'noun': 'Điện_thoại',
  'noun_index': 1,
  'attribute': 'hợp_lý',
  'attribute_index': 3,
  'attribute_pos': 'A',
  'modifiers': []},
 {'noun': 'chất_lượng',
  'noun_index': 5,
  'attribute': 'tốt',
  'attribute_index': 6,
  'attribute_pos': 'A',
  'modifiers': []}]

In [20]:
print(example5)
print("\n---\n")
extract_attributes_from_all_segments_of_sentence(example5)

nhân viên thân thiện tư vấn nhiệt tình, cảm ơn nhân viên đmx Quảng Tùng

---



[{'noun': 'nhân_viên',
  'noun_index': 1,
  'attribute': 'thân_thiện',
  'attribute_index': 2,
  'attribute_pos': 'A',
  'modifiers': []}]

In [21]:
print(example6)
print("\n---\n")
extract_attributes_from_all_segments_of_sentence(example6)

sản phẩm sài ok lắm

---



[{'noun': 'sản_phẩm',
  'noun_index': 1,
  'attribute': 'ok',
  'attribute_index': 3,
  'attribute_pos': 'A',
  'modifiers': []}]

In [22]:
print(example7)
print("\n---\n")
extract_attributes_from_all_segments_of_sentence(example7)

điện thoại dùng mượt, chụp hình đẹp, pin dùng mãi chả thấy hết để sạc, rất hài lòng. Ai đang định mua thì chọn luôn không phải lăn tăn nhé

---



[{'noun': 'điện_thoại',
  'noun_index': 1,
  'attribute': 'mượt',
  'attribute_index': 3,
  'attribute_pos': 'A',
  'modifiers': []},
 {'noun': 'hình',
  'noun_index': 6,
  'attribute': 'đẹp',
  'attribute_index': 7,
  'attribute_pos': 'A',
  'modifiers': []}]

In [23]:
example8 = "Nhân viên giao hàng nhiệt tình, sản phẩm chất lượng tốt, giá cả hợp lý."
print(example8)
print("\n---\n")
extract_attributes_from_all_segments_of_sentence(example8)

Nhân viên giao hàng nhiệt tình, sản phẩm chất lượng tốt, giá cả hợp lý.

---



[{'noun': 'hàng',
  'noun_index': 3,
  'attribute': 'nhiệt_tình',
  'attribute_index': 4,
  'attribute_pos': 'A',
  'modifiers': []},
 {'noun': 'sản_phẩm',
  'noun_index': 6,
  'attribute': 'tốt',
  'attribute_index': 8,
  'attribute_pos': 'A',
  'modifiers': []},
 {'noun': 'giá_cả',
  'noun_index': 10,
  'attribute': 'hợp_lý',
  'attribute_index': 11,
  'attribute_pos': 'A',
  'modifiers': []}]